In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from SALib.sample.morris.strategy import Strategy
from SALib.sample.morris import generate_trajectory, generate_trajectory_2, _sample_oat, \
generate_p_star, generate_x_star, generate_x_star_2, compute_b_star, compute_b_star_2, compute_delta

In [2]:
sample = _sample_oat({'num_vars': 10}, 5)

In [3]:
sample.shape

(55, 10)

In [4]:
sample[:11, 4]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.33333333, 0.33333333,
       0.33333333])

## `sample` matrix

The way that this array is constructed is... unintuituve. Maybe it is like this for backwards compatibility. `sample` is a two-dimensional array which includes three dimensions: trajectory, parameter, and parameter value. Personally, I would prefer this to be a three dimensional array (order doesn't really matter).

Note: My (older, unmaintained) [py-elem-effects](https://bitbucket.org/cmutel/py-elem-effects/) library doesn't store the coordinates of every single point at each point on the trajectory, but rather the starting and ending values for each parameter, and the order the parameters change in. This means you reduce memory usage substantially.

The rows are parameter values for the first trajectory, follow by for the second trajectory, etc. Columns are the parameters.

## Trajectory generation performance

The function `_sample_oat` includes this line:

    sample = np.array([generate_trajectory(group_membership, num_levels)
                       for n in range(N)])

These types of loops should almost always be avoided, as there are numpy operators that are much faster. Let's see if we can make trajectory generation faster.

In [5]:
m = np.asmatrix(np.identity(1000, dtype=int))

In [6]:
m.shape

(1000, 1000)

In [7]:
%timeit generate_trajectory(m, 4)

65.6 ms ± 801 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
type(generate_trajectory(m, 4))

numpy.matrixlib.defmatrix.matrix

In [9]:
%timeit generate_trajectory_2(m, 4)

55.6 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
type(generate_trajectory_2(m, 4))

numpy.matrixlib.defmatrix.matrix

In [13]:
import numpy.random as rd
rd.seed(5)

b1 = generate_trajectory(m, 4)
b2 = generate_trajectory_2(m, 4)

In [ ]:
b1

In [ ]:
b2

In [ ]:
np.all(b1==b2)